In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'C:\Users\pierr\Documents\R\Quarto-test':
  os.chdir(r'C:\Users\pierr\Documents\R\Quarto-test')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


{"C:\\Users\\pierr\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\importlib\\_bootstrap.py": 1680645850.0, "C:\\Users\\pierr\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\importlib\\_bootstrap_external.py": 1680645850.0, "C:\\Users\\pierr\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\zipimport.py": 1680645850.0, "C:\\Users\\pierr\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\codecs.py": 1680645850.0, "C:\\Users\\pierr\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\encodings\\aliases.py": 1680645850.0, "C:\\Users\\pierr\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\encodings\\__init__.py": 1680645850.0, "C:\\Users\\pierr\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\encodings\\utf_8.py": 1680645850.0, "C:\\Users\\pierr\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\abc.py": 1680645850.0, "C:\\Users\\pierr\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\io.py": 1680645850.0, "C:\\Users\\pierr\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\stat.

In [2]:
import logging
import re
import requests
import csv
from bs4 import BeautifulSoup


def get(uri):
    logging.info(f'Will query: {uri}')
    headers = {'User-Agent': 'Mozilla/5.0'}
    r = requests.get(uri, headers=headers)
    r.raise_for_status()
    r.encoding = 'UTF-8'
    return r


def get_soup(uri: str) -> BeautifulSoup:
    r = get(uri)
    return BeautifulSoup(r.text, 'html.parser')

def main():
    host = "https://www.sephora.fr"
    path = "/shop/cheveux/soin-cheveux/shampoing-c447/"
    uri = host + path
    soup = get_soup(uri)

    with open('sephora_data.csv', 'w', newline='\n',encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['ingredients', 'brand', 'name', 'price', 'quantity', 'unit'])


        expected_article_amount = soup.find('div', {'class': 'results-hits'}).text.split()[0]
        logging.info(f'Will get {expected_article_amount} of articles.')
        all_articles_uri = f'{host}{path}?srule=GoLiveSortingRule&sz={expected_article_amount}&format=page-element&start=0&on=onclickload'
        soup = get_soup(all_articles_uri)
        articles = soup.find_all('div', attrs={'class': 'product-info-wrapper'})
        logging.info(f'Got {len(articles)} articles.')
        for article in articles:
            article_product_link = article.find('a', attrs={'class': 'product-tile-link'} )['href']
            article_soup = get_soup(article_product_link)
            brand = article_soup.find('a', {'class': 'brand-link'}).text.replace('\n', '')
            name = article_soup.find('span', {'class': 'product-name product-name-bold'}).text.replace('\n', '')
            try:
                ingredients = article_soup.find('div', {'class': 'accordion-content ingredients-content'}).text.replace('\n','').split('.')[0]
                ingredients = re.sub("[\(\[].*?[\)\]]", "", ingredients)
            except Exception as e:
                logging.error("Couldn't find the ingredients")
                continue
            try:
                # unit contains the price and price the unit (go figure, I guess they are not talented enough)
                price = article_soup.find('span', {'class': 'unit'}).text.replace('\n', '').replace(' €', '')
                quantity, unit = article_soup.find('span', {'class': 'price'}).text.replace('\n', '').replace('ml', ' ml').split()
            except Exception as e:
                logging.error("Couldn't find the price or unit ")
                continue

            writer.writerow([ingredients, brand, name, price, quantity, unit])


if __name__ == '__main__':
    logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
    main()

INFO:Will query: https://www.sephora.fr/shop/cheveux/soin-cheveux/shampoing-c447/


INFO:Will get 339 of articles.


INFO:Will query: https://www.sephora.fr/shop/cheveux/soin-cheveux/shampoing-c447/?srule=GoLiveSortingRule&sz=339&format=page-element&start=0&on=onclickload


INFO:Got 339 articles.


INFO:Will query: https://www.sephora.fr/p/scalp-revival-charcoal---shampooing-exfoliant-441218.html


...

INFO:Will query: https://www.sephora.fr/p/absolut-repair---shampoing-reparateur-et-hydratant-pour-cheveux-abimes-557761.html


INFO:Will query: https://www.sephora.fr/p/fusion-intense-repair---shampoing-reparation-intense-pour-cheveux-abimes-444299.html


INFO:Will query: https://www.sephora.fr/p/vitamino-color---shampooing-fixateur-de-couleur-antioxydant-et-filtre-uv-557776.html


INFO:Will query: https://www.sephora.fr/p/pro-longer---shampoing-pour-cheveux-longs-aux-pointes-afffinees-557770.html


INFO:Will query: https://www.sephora.fr/p/serie-expert---blondifier-shampoo-cool-557765.html
